Run MLP (first layer weights fixed) on mnist and compute bias and variance

In [1]:
%matplotlib inline
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import queue
import os
import sys
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import copy 
import pandas as pd
class MLP(nn.Module):
    def __init__(self, p, d, o):
        """RF_models
        
        Args:
            p (int): the hidden size
            d (int): the input feature dimension
            o (int): the output dimension
        """
        super(MLP, self).__init__()

        self.fc1 = nn.Linear(d, p, bias=False)
        self.fc2 = nn.Linear(p, o, bias=False)
        self.p = p
        self.d = d
        self.o = o 
    def forward(self, x):
        o = F.relu(self.fc1(x))
        o = self.fc2(o)
        return o
class Ensemble_Two_Layer_NN(object):
    def __init__(self, n_classifiers, p, d=784, o=10):
        """Ensemble_Two_Layer_NN
        
        Args:
            p (int): the hidden size
            d (int, optional): the input feature dimension
            o (int, optional): the output dimension
            coef (float, optional): the ridge regression penalty coefficient
        """
        self.n_classifiers = n_classifiers
        self.p = p
        self.d = d 
        self.o = o 
        self.coef = coef
        self.learners = queue.LifoQueue(maxsize = self.n_classifiers)
        self.MODEL_TYPE = MLP
    def __len__(self):
        return len(self.learners.queue)
    
    def put_model_rho(self, model, rho):
        self.learners.put([model, rho])
    def get_init_model(self, cuda=True):
        model = self.MODEL_TYPE(self.p, self.d, self.o)
        if cuda:
            model.cuda()
        return model
    def cuda(self):
        if len(self) == 0:
            return 
        else:
            for model, rho in self.learners.queue:
                model.cuda()
            return
    def train(self):
        if len(self)!=0:
            for model, rho in self.learners.queue:
                model.train()
    def eval(self):
        if len(self)!=0:
            for model, rho in self.learners.queue:
                model.eval()
    def forward(self, x):
        Bs = x.size(0)
        if len(self) == 0:
            zeros = torch.zeros(Bs, self.o)
            zeros = zeros.to(x.device)
            return zeros
        else:
            outputs = torch.zeros(Bs, self.o)
            outputs = outputs.to(x.device) 
            for model, rho in self.learners.queue:
                output = model(x)
                outputs += rho*output
            return outputs


In [2]:
def get_subsample_dataset(trainset, subset):
    trainsubset = copy.deepcopy(trainset)
    trainsubset.data = [trainsubset.data[index] for index in subset]
    trainsubset.targets = [trainsubset.targets[index] for index in subset]
    return trainsubset
def fix_width_number(width, n_classifiers):
    return max(1, width//n_classifiers)

# Training
def train(net, trainset, permute_index, train_size, num_iters, lr, batch_size, coef):
    net.train()
    subsample_indexes = np.random.choice(permute_index, size=train_size)
    trainsubset = get_subsample_dataset(trainset, subsample_indexes)
    trainloader = torch.utils.data.DataLoader(trainsubset, batch_size=batch_size, shuffle=True)

    for i_c in range(net.n_classifiers):
        i_iter = 0
        model = net.get_init_model(cuda=True)
        rho = 1/net.n_classifiers
        optimizer = torch.optim.SGD(model.fc2.parameters(), lr=lr)
        lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size = num_iters//3, gamma = 0.1)
        while i_iter < num_iters:
            for inputs, targets in trainloader:
                Bs = inputs.size(0)
                inputs = inputs.reshape(Bs, -1)
                inputs, targets = inputs.cuda(), targets.cuda()
                targets_onehot = torch.FloatTensor(targets.size(0), net.o).cuda()
                targets_onehot.zero_()
                targets_onehot.scatter_(1, targets.view(-1, 1).long(), 1)
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, targets_onehot)
                l2_reg = 0
                for param in model.fc2.parameters():
                    l2_reg += coef * torch.norm(param)
                mse_loss = loss.item()
                l2_loss = l2_reg.item()
                loss += l2_reg
                loss.backward()
                optimizer.step()
                lr_scheduler.step()
                string = "Train {} model: Iters [{}/{}] mse: {:.4f}, l2_loss: {:.4f}, train_loss:{:.4f}".format(i_c+1, i_iter, num_iters,  mse_loss, l2_loss, loss.item())
                sys.stdout.write(string+"\r")
                sys.stdout.flush()
                i_iter +=1
        net.put_model_rho(model, rho)
    # after all models were trained, estimate the mse error
    train_loss = 0
    correct = 0
    total = 0
    for inputs, targets in trainloader:
        Bs = inputs.size(0)
        inputs = inputs.reshape(Bs, -1)
        inputs, targets = inputs.cuda(), targets.cuda()
        targets_onehot = torch.FloatTensor(targets.size(0), net.o).cuda()
        targets_onehot.zero_()
        targets_onehot.scatter_(1, targets.view(-1, 1).long(), 1)
        outputs = net.forward(inputs)
        loss = criterion(outputs, targets_onehot)
        train_loss = loss.item() * outputs.numel()
        _, predicted = outputs.max(1)
        correct = predicted.eq(targets).sum().item()
        total = targets.size(0)
    return train_loss/ total , 100. * correct / total

# Test
def test(net, testloader):
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.cuda(), targets.cuda()
            Bs = inputs.size(0)
            inputs = inputs.reshape(Bs, -1)
            targets_onehot = torch.FloatTensor(targets.size(0), net.o).cuda()
            targets_onehot.zero_()
            targets_onehot.scatter_(1, targets.view(-1, 1).long(), 1)
            outputs = net.forward(inputs)
            loss = criterion(outputs, targets_onehot)
            test_loss += loss.item() * outputs.numel()
            _, predicted = outputs.max(1)
            correct += predicted.eq(targets).sum().item()
            total += targets.size(0)
    return test_loss / total, 100. * correct / total

def compute_bias_variance(net, testloader, trial, OUTPUST_SUM, OUTPUTS_SUMNORMSQUARED):
    net.eval()
    bias2 = 0
    variance = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.cuda(), targets.cuda()
            Bs = inputs.size(0)
            inputs = inputs.reshape(Bs, -1)
            targets_onehot = torch.FloatTensor(targets.size(0), net.o).cuda()
            targets_onehot.zero_()
            targets_onehot.scatter_(1, targets.view(-1, 1).long(), 1)
            outputs = net.forward(inputs)
            OUTPUST_SUM[total:(total + targets.size(0)), :] += outputs
            OUTPUTS_SUMNORMSQUARED[total:total + targets.size(0)] += outputs.norm(dim=1) ** 2.0

            bias2 += (OUTPUST_SUM[total:total + targets.size(0), :] / (trial + 1) - targets_onehot).norm() ** 2.0
            variance += OUTPUTS_SUMNORMSQUARED[total:total + targets.size(0)].sum()/(trial + 1) - (OUTPUST_SUM[total:total + targets.size(0), :]/(trial + 1)).norm() ** 2.0
            total += targets.size(0)

    return bias2 / total, variance / total


In [3]:
transform_train = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])
trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform_train)
testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=4)
# loss definition
criterion = nn.MSELoss(reduction='mean').cuda()

def run_exps_sgd(train_sizes, N_Ds, P_Ns, trainset, test_size, feature_dim, num_classes, num_trials, coef,
             outdir, save_csv, num_iters, lr, batch_size_list, K = 1):
    df = pd.DataFrame()
    for batch_size in batch_size_list:
        for train_size in train_sizes:
            hidden_sizes = P_Ns * train_size
            hidden_sizes = np.unique([int(np.around(x)) for x in hidden_sizes])
            for hidden_size in hidden_sizes:
                TRAIN_ACC_SUM = 0.0
                TEST_ACC_SUM = 0.0
                TRAIN_LOSS_SUM = 0.0
                TEST_LOSS_SUM = 0.0
                permute_index = np.random.permutation(len(trainset))
                OUTPUST_SUM = torch.Tensor(test_size, num_classes).zero_().cuda()
                OUTPUTS_SUMNORMSQUARED = torch.Tensor(test_size).zero_().cuda()
                for trial in range(num_trials):
                    net = Ensemble_Two_Layer_NN(n_classifiers = K, p = fix_width_number(hidden_size, K), d=feature_dim, o=num_classes)
                    net.cuda()
                    train_loss, train_acc = train(net, trainset, permute_index, train_size,
                                                 num_iters, lr, batch_size, coef)
                    test_loss, test_acc = test(net, testloader)

                    TRAIN_LOSS_SUM += train_loss
                    TEST_LOSS_SUM += test_loss
                    TRAIN_ACC_SUM += train_acc
                    TEST_ACC_SUM += test_acc

                    # compute bias and variance
                    bias2, variance = compute_bias_variance(net, testloader, trial, OUTPUST_SUM, OUTPUTS_SUMNORMSQUARED)
                    variance_unbias = variance * num_trials / (num_trials - 1.0)
                    bias2_unbias = TEST_LOSS_SUM / (trial + 1) - variance_unbias
                    print('Train size: [{}] hidden size: [{}] batch size: [{}] trial: {}, train_loss: {:.6f}, train acc: {}, test loss: {:.6f}, test acc: {}, bias2: {}, variance: {}'.format(
                        train_size, hidden_size, batch_size,
                        trial, TRAIN_LOSS_SUM / (trial + 1), TRAIN_ACC_SUM / (trial + 1), TEST_LOSS_SUM / (trial + 1),
                        TEST_ACC_SUM / (trial + 1), bias2_unbias, variance_unbias))
                    torch.cuda.empty_cache()
                print('#'*50)
                df = df.append({'train_size': train_size, 'hidden_size':hidden_size, 'batch_size': batch_size,
                                'train_loss': TRAIN_LOSS_SUM / (trial + 1), 'train_acc': TRAIN_ACC_SUM / (trial + 1),
                                'test_loss': TEST_LOSS_SUM / (trial + 1), 'test_acc': TEST_ACC_SUM / (trial + 1), 
                               'variance': variance_unbias.item(),
                               'bias2': bias2_unbias.item()}, ignore_index=True)
                df.to_csv(os.path.join(outdir, save_csv))
    df.to_csv(os.path.join(outdir, save_csv))

In [ ]:
num_classes = 10
num_trials = 50
coef = 0.0001
N_Ds = [1]
feature_dim = 784
lr = 0.01 
batch_size_list = [10, 784]
train_sizes = [int(np.around(x*feature_dim)) for x in N_Ds]
test_size = 10000
P_Ns = 10** np.linspace(-2, 1, 50)

for num_iters in [500, 5000]:
    outdir = 'mnist_SGD/num_iters_{}_coef={}'.format(num_iters, coef)
    print(outdir)
    if not os.path.exists(outdir):
        os.makedirs(outdir)
    run_exps_sgd(train_sizes, N_Ds, P_Ns, trainset, test_size, feature_dim, num_classes, num_trials, coef,
             outdir, 'singleNN_output.csv', num_iters = num_iters, lr=lr, batch_size_list=batch_size_list,  K = 1)
    run_exps_sgd(train_sizes, N_Ds, P_Ns, trainset, test_size, feature_dim, num_classes, num_trials, coef,
                 outdir, 'ensembleNNK=2_output.csv', num_iters = num_iters, lr=lr, batch_size_list=batch_size_list,  K = 2)

mnist_SGD/num_iters_500_coef=0.0001
Train size: [784] hidden size: [8] batch size: [10] trial: 0, train_loss: 1.082811, train acc: 0.0, test loss: 1.151223, test acc: 17.67, bias2: 1.1512227058410645, variance: -2.919411101753866e-10
Train size: [784] hidden size: [8] batch size: [10] trial: 1, train_loss: 1.261458, train acc: 0.0, test loss: 1.295968, test acc: 14.780000000000001, bias2: 1.1295108795166016, variance: 0.1664571762084961
Train size: [784] hidden size: [8] batch size: [10] trial: 2, train_loss: 1.307112, train acc: 0.0, test loss: 1.321589, test acc: 14.57, bias2: 1.0829147100448608, variance: 0.23867428302764893
Train size: [784] hidden size: [8] batch size: [10] trial: 3, train_loss: 1.294072, train acc: 0.0, test loss: 1.340458, test acc: 12.342500000000001, bias2: 1.0593767166137695, variance: 0.28108102083206177
Train size: [784] hidden size: [8] batch size: [10] trial: 4, train_loss: 1.383258, train acc: 0.0, test loss: 1.359596, test acc: 12.16, bias2: 1.058304071

Train size: [784] hidden size: [8] batch size: [10] trial: 38, train_loss: 1.298965, train acc: 5.769230769230769, test loss: 1.342376, test acc: 10.10282051282051, bias2: 0.978598952293396, variance: 0.3637765944004059
Train size: [784] hidden size: [8] batch size: [10] trial: 39, train_loss: 1.292434, train acc: 6.25, test loss: 1.341078, test acc: 10.113499999999998, bias2: 0.9763325452804565, variance: 0.36474499106407166
Train size: [784] hidden size: [8] batch size: [10] trial: 40, train_loss: 1.292440, train acc: 6.097560975609756, test loss: 1.338911, test acc: 10.12341463414634, bias2: 0.9764930009841919, variance: 0.3624175786972046
Train size: [784] hidden size: [8] batch size: [10] trial: 41, train_loss: 1.288635, train acc: 5.9523809523809526, test loss: 1.335548, test acc: 9.989047619047616, bias2: 0.9767924547195435, variance: 0.3587553799152374
Train size: [784] hidden size: [8] batch size: [10] trial: 42, train_loss: 1.285624, train acc: 5.813953488372093, test loss: 1

Train size: [784] hidden size: [9] batch size: [10] trial: 26, train_loss: 1.278606, train acc: 12.962962962962964, test loss: 1.297617, test acc: 10.40037037037037, bias2: 0.936322808265686, variance: 0.361294150352478
Train size: [784] hidden size: [9] batch size: [10] trial: 27, train_loss: 1.284965, train acc: 13.392857142857142, test loss: 1.292711, test acc: 10.4225, bias2: 0.9353330135345459, variance: 0.35737839341163635
Train size: [784] hidden size: [9] batch size: [10] trial: 28, train_loss: 1.277033, train acc: 12.931034482758621, test loss: 1.288423, test acc: 10.539310344827586, bias2: 0.9317633509635925, variance: 0.3566599488258362
Train size: [784] hidden size: [9] batch size: [10] trial: 29, train_loss: 1.278571, train acc: 12.5, test loss: 1.285225, test acc: 10.452, bias2: 0.9317762851715088, variance: 0.35344839096069336
Train size: [784] hidden size: [9] batch size: [10] trial: 30, train_loss: 1.287623, train acc: 12.096774193548388, test loss: 1.291310, test acc:

Train size: [784] hidden size: [10] batch size: [10] trial: 14, train_loss: 1.298010, train acc: 8.333333333333334, test loss: 1.306428, test acc: 11.213333333333335, bias2: 0.9733267426490784, variance: 0.33310168981552124
Train size: [784] hidden size: [10] batch size: [10] trial: 15, train_loss: 1.312029, train acc: 7.8125, test loss: 1.306842, test acc: 11.02875, bias2: 0.9749418497085571, variance: 0.33189988136291504
Train size: [784] hidden size: [10] batch size: [10] trial: 16, train_loss: 1.342984, train acc: 7.352941176470588, test loss: 1.321200, test acc: 11.030000000000001, bias2: 0.9716753959655762, variance: 0.3495241105556488
Train size: [784] hidden size: [10] batch size: [10] trial: 17, train_loss: 1.349364, train acc: 6.944444444444445, test loss: 1.319679, test acc: 11.096666666666668, bias2: 0.9665683507919312, variance: 0.3531111776828766
Train size: [784] hidden size: [10] batch size: [10] trial: 18, train_loss: 1.344174, train acc: 6.578947368421052, test loss: 

Train size: [784] hidden size: [12] batch size: [10] trial: 2, train_loss: 1.208296, train acc: 0.0, test loss: 1.232251, test acc: 9.909999999999998, bias2: 1.0399352312088013, variance: 0.19231536984443665
Train size: [784] hidden size: [12] batch size: [10] trial: 3, train_loss: 1.185280, train acc: 0.0, test loss: 1.230808, test acc: 9.889999999999999, bias2: 1.032089352607727, variance: 0.19871844351291656
Train size: [784] hidden size: [12] batch size: [10] trial: 4, train_loss: 1.169298, train acc: 5.0, test loss: 1.258882, test acc: 9.291999999999998, bias2: 1.0390169620513916, variance: 0.21986529231071472
Train size: [784] hidden size: [12] batch size: [10] trial: 5, train_loss: 1.161824, train acc: 8.333333333333334, test loss: 1.265059, test acc: 9.336666666666666, bias2: 1.0268676280975342, variance: 0.2381916642189026
Train size: [784] hidden size: [12] batch size: [10] trial: 6, train_loss: 1.215494, train acc: 7.142857142857143, test loss: 1.257815, test acc: 9.01714285

...

Train size: [784] hidden size: [86] batch size: [784] trial: 31, train_loss: 0.990162, train acc: 14.963329081632656, test loss: 0.996992, test acc: 14.214062499999999, bias2: 0.8674598932266235, variance: 0.12953227758407593
Train size: [784] hidden size: [86] batch size: [784] trial: 32, train_loss: 0.990910, train acc: 14.892547928262216, test loss: 0.998020, test acc: 14.10121212121212, bias2: 0.8678715825080872, variance: 0.13014839589595795
Train size: [784] hidden size: [86] batch size: [784] trial: 33, train_loss: 0.989473, train acc: 15.039765906362547, test loss: 0.996437, test acc: 14.308235294117646, bias2: 0.8662769794464111, variance: 0.13015970587730408
Train size: [784] hidden size: [86] batch size: [784] trial: 34, train_loss: 0.988378, train acc: 15.160349854227407, test loss: 0.995085, test acc: 14.474285714285713, bias2: 0.8643815517425537, variance: 0.1307038962841034
Train size: [784] hidden size: [86] batch size: [784] trial: 35, train_loss: 0.988978, train acc: 

Train size: [784] hidden size: [99] batch size: [784] trial: 18, train_loss: 0.981781, train acc: 15.238990332975295, test loss: 0.984998, test acc: 14.901052631578947, bias2: 0.8617306351661682, variance: 0.12326780706644058
Train size: [784] hidden size: [99] batch size: [784] trial: 19, train_loss: 0.980133, train acc: 15.293367346938775, test loss: 0.983475, test acc: 14.91, bias2: 0.8604713678359985, variance: 0.123003289103508
Train size: [784] hidden size: [99] batch size: [784] trial: 20, train_loss: 0.980689, train acc: 15.184645286686102, test loss: 0.984734, test acc: 14.762857142857142, bias2: 0.8614701628684998, variance: 0.12326370924711227
Train size: [784] hidden size: [99] batch size: [784] trial: 21, train_loss: 0.981784, train acc: 15.103200371057513, test loss: 0.985915, test acc: 14.736818181818181, bias2: 0.8625028133392334, variance: 0.12341216206550598
Train size: [784] hidden size: [99] batch size: [784] trial: 22, train_loss: 0.982000, train acc: 15.2395740905

Train size: [784] hidden size: [114] batch size: [784] trial: 5, train_loss: 0.939430, train acc: 18.792517006802722, test loss: 0.946086, test acc: 18.976666666666667, bias2: 0.8393636345863342, variance: 0.10672269016504288
Train size: [784] hidden size: [114] batch size: [784] trial: 6, train_loss: 0.943023, train acc: 18.476676384839653, test loss: 0.951100, test acc: 18.157142857142855, bias2: 0.8414565920829773, variance: 0.10964374244213104
Train size: [784] hidden size: [114] batch size: [784] trial: 7, train_loss: 0.945823, train acc: 18.03252551020408, test loss: 0.954305, test acc: 17.64875, bias2: 0.8421844244003296, variance: 0.11212043464183807
Train size: [784] hidden size: [114] batch size: [784] trial: 8, train_loss: 0.944057, train acc: 18.494897959183675, test loss: 0.952363, test acc: 17.962222222222223, bias2: 0.8387810587882996, variance: 0.113582082092762
Train size: [784] hidden size: [114] batch size: [784] trial: 9, train_loss: 0.944670, train acc: 18.55867346

Train size: [784] hidden size: [114] batch size: [784] trial: 42, train_loss: 0.963497, train acc: 16.709183673469386, test loss: 0.968283, test acc: 16.308837209302332, bias2: 0.8412360548973083, variance: 0.12704670429229736
Train size: [784] hidden size: [114] batch size: [784] trial: 43, train_loss: 0.963337, train acc: 16.712082560296846, test loss: 0.968220, test acc: 16.300000000000008, bias2: 0.8410045504570007, variance: 0.1272156983613968
Train size: [784] hidden size: [114] batch size: [784] trial: 44, train_loss: 0.964119, train acc: 16.621315192743765, test loss: 0.969005, test acc: 16.215777777777785, bias2: 0.8415286540985107, variance: 0.12747672200202942
Train size: [784] hidden size: [114] batch size: [784] trial: 45, train_loss: 0.964226, train acc: 16.570541259982257, test loss: 0.969272, test acc: 16.13456521739131, bias2: 0.8417400121688843, variance: 0.12753239274024963
Train size: [784] hidden size: [114] batch size: [784] trial: 46, train_loss: 0.964065, train 

Train size: [784] hidden size: [131] batch size: [784] trial: 29, train_loss: 0.962860, train acc: 17.389455782312925, test loss: 0.968310, test acc: 16.431000000000004, bias2: 0.8430306315422058, variance: 0.12527979910373688
Train size: [784] hidden size: [131] batch size: [784] trial: 30, train_loss: 0.961211, train acc: 17.416886109282423, test loss: 0.966596, test acc: 16.552903225806453, bias2: 0.8414320945739746, variance: 0.12516441941261292
Train size: [784] hidden size: [131] batch size: [784] trial: 31, train_loss: 0.961049, train acc: 17.44658801020408, test loss: 0.966390, test acc: 16.585625000000004, bias2: 0.8412340879440308, variance: 0.12515562772750854
Train size: [784] hidden size: [131] batch size: [784] trial: 32, train_loss: 0.960951, train acc: 17.350803957946816, test loss: 0.966017, test acc: 16.54484848484849, bias2: 0.8410612344741821, variance: 0.1249561682343483
Train size: [784] hidden size: [131] batch size: [784] trial: 33, train_loss: 0.960392, train a

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
font = {
        'size'   : 18}
matplotlib.rc('font', **font)
figsize = (16, 5)
import seaborn as sns
sns.set_style('darkgrid')
import pandas as pd

def plot_bias_var(df, N_D, ymin=0, ymax=1.0):
    fig1, axes1 = plt.subplots(1, 3, figsize=figsize)
    axes1[0].set_xscale('log')
    axes1[1].set_xscale('log')
    axes1[2].set_xscale('log')
    cur_df = df[df['train_size']/feature_dim==N_D]
    test_loss = cur_df['test_loss']
    bias2 = cur_df['bias2']
    var = cur_df['variance']
    P_N = cur_df['hidden_size']/cur_df['train_size']
    axes1[0].plot(P_N, test_loss)
    axes1[0].set_xlabel("P/N")
    axes1[0].set_ylabel("Test Loss")
    axes1[0].set_ylim(ymin, ymax)
    axes1[1].plot(P_N, bias2)
    axes1[1].set_xlabel("P/N")
    axes1[1].set_ylabel("Bias Square")
    axes1[1].set_ylim(ymin, ymax)
    axes1[2].plot(P_N, var)
    axes1[2].set_xlabel("P/N")
    axes1[2].set_ylabel("Variance")
    axes1[2].set_ylim(ymin, ymax)
    fig1.suptitle("Bias-Variance Decomposition (N/D={:.2f})".format(N_D))
    plt.show()
def plot_single_vs_ensemble(dfs_list, Ks_list, N_D, feature_dim, ymin=0, ymax=1.0):
    assert len(dfs_list) == len(Ks_list)
    fig1, axes1 = plt.subplots(1, 3, figsize=figsize)
    for i in range(3):
        axes1[i].set_xscale('log')
    dfs_list = [df[df['train_size']/feature_dim==N_D] for df in dfs_list]
    for cur_df, K in zip(dfs_list, Ks_list):
        test_loss = cur_df['test_loss']
        bias2 = cur_df['bias2']
        var = cur_df['variance']
        P_N = cur_df['hidden_size']/cur_df['train_size']
        axes1[0].plot(P_N, test_loss, label='K={}'.format(K))
        axes1[1].plot(P_N, bias2, label='K={}'.format(K))
        axes1[2].plot(P_N, var, label='K={}'.format(K))
    
    axes1[0].set_xlabel("P/N")
    axes1[0].set_ylabel("Test Loss")
    axes1[0].set_ylim(ymin, ymax)
    
    axes1[1].set_xlabel("P/N")
    axes1[1].set_ylabel("Bias Square")
    axes1[1].set_ylim(ymin, ymax)
    
    axes1[2].set_xlabel("P/N")
    axes1[2].set_ylabel("Variance")
    axes1[2].set_ylim(ymin, ymax)
    fig1.suptitle("Bias-Variance Decomposition (N/D={:.2f})".format(N_D))
    plt.legend()
    plt.show()


In [ ]:
# K2_df = pd.read_csv(os.path.join(outdir, 'ensembleNNK=2_output.csv'))
# K1_df = pd.read_csv(os.path.join(outdir, 'singleNN_output.csv'))
# plot_single_vs_ensemble([K1_df, K2_df], [1, 2], N_Ds[0], 784,)